[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/stable-cascade-jupyter/blob/main/stable_cascade_gradio_jupyter.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/stable-cascade-hf
%cd /content/stable-cascade-hf

!wget https://huggingface.co/spaces/multimodalart/stable-cascade/resolve/main/previewer/previewer_v1_100k.pt -O /content/stable-cascade-hf/previewer/previewer_v1_100k.pt
!wget https://huggingface.co/spaces/multimodalart/stable-cascade/resolve/main/previewer/text2img_wurstchen_b_v1_previewer_100k.pt -O /content/stable-cascade-hf/previewer/text2img_wurstchen_b_v1_previewer_100k.pt

!pip install -q git+https://github.com/kashif/diffusers.git@diffusers-yield-callback
!pip install -q gradio accelerate

!python app.py